## Extraction of datasets and data synthesis required for Car Price Analysis

#### The following datasets were scraped from the web. These datasets may have important pointers for price and price category prediction. This notebook describes how these datasets are augmented to produce a combined data set.
    

<div class="alert alert-block alert-info">



| Data Set Type | Data Source | Fields of Importance | Keys for join | Data Frame Name
| --- | --- | --- | --- | ---
| Actual Car Data listing Extraction | www.truecar.com | All | All | df_cardata
| Car Categories based on Make and Model | https://www.back4app.com | Category | Year, Make, Model  | df_category
| Car Reliability Index by Make | https://www.usatoday.com | Brand (Make) | BrandReliabilityRank | df_reliability
| Cost of living index for each city | https://meric.mo.gov | Cost of Living Index, Transportation Index | City | df_cost
| % of market sales in US by make | https://www.goodcarbadcar.net | PercentSales | Make | df_sales 
| "Days to turn" for used cars by make | https://www.edmunds.com | AvgDaysToTurn | Make | df_turn

</div>

In [1]:
import pandas as pd
import json
import requests
import numpy as np
requests.packages.urllib3.disable_warnings() 
pd.set_option('display.max_colwidth',None)
pd.set_option('display.float_format',lambda x: '%.2f' %x)
pd.set_option('display.max_rows',300)
pd.set_option('display.max_columns',None)


# Extraction of raw data

### Data Set 1 :  Main car data listing.

Please refer to file _WebScrapeCarData.py_ for the scraping code. Here, the csv file created after extraction is read for data augmentation

<span style="color:red"> #### Dataframe = df_cardata </span>

In [2]:
# Get main car details data

df_cardata = pd.read_csv("cardata.csv",delimiter = '|',index_col = False, encoding='cp1252')
for col in ['make','model','trim','pricecategory','city','state','colorexterior','colorinterior','usage']:
    df_cardata[col] = df_cardata[col].astype(str).apply(lambda x:x.upper())
df_cardata['discount'] = df_cardata['pricevariance'].astype(str).apply(lambda x:'Y' if 'off' in x.lower() else 'N')
df_cardata.drop(columns = ['pricevariance'],inplace = True)
df_cardata['model'] = df_cardata['model'].astype(str).apply(lambda x:x.replace('-',''))
df_cardata['owner'].fillna(0,inplace = True)
df_cardata['owner'] = df_cardata['owner'].astype(int)
df_cardata['accidenthist'] = df_cardata['accidenthist'].apply(lambda x:0 if x.strip() == 'no' else x)
df_cardata['accidenthist'] = df_cardata['accidenthist'].astype(int)
df_cardata['trim']
df_cardata.head()
# 9996 records

,vin,year,make,model,trim,pricecategory,price,mileage,city,state,colorexterior,colorinterior,accidenthist,owner,usage,discount
0,WBY1Z2C51FV286674,2015,BMW,I3,60 AH,FAIR PRICE,15991,21493,BELLEVUE,WA,SILVER,GRAY,1,2,PERSONAL,N
1,2GNAXHEV4J6220616,2018,CHEVROLET,EQUINOX,LS WITH 1LS FWD,FAIR PRICE,14899,37071,NORCO,CA,BLACK,GRAY,0,1,PERSONAL,Y
2,4S3GTAD6XK3741106,2019,SUBARU,IMPREZA,2.0I PREMIUM 5-DOOR CVT,FAIR PRICE,19220,15914,STAFFORD,TX,RED,BEIGE,0,1,PERSONAL,N
3,2C4RDGCG8KR551301,2019,DODGE,GRAND CARAVAN,SXT,EXCELLENT PRICE,12993,42070,OCALA,FL,GRAY,BLACK,0,1,PERSONAL,Y
4,5YFEPRAEXLP047434,2020,TOYOTA,COROLLA,LE CVT,EXCELLENT PRICE,13800,18725,BOERNE,TX,WHITE,BLACK,0,1,PERSONAL,N


### Data Set 2 :  Car Categories based on Make and Mode

<span style="color:red"> #### Dataframe = df_category </span>

In [3]:
# get car category data

# url = 'https://parseapi.back4app.com/classes/Car_Model_List?limit=5000'
# headers = {
#     'X-Parse-Application-Id': 'hlhoNKjOvEhqzcVAJ1lxjicJLZNVv36GdbboZj3Z', # This is the fake app's application id
#     'X-Parse-Master-Key': 'SNMJJF0CZZhTPhLDIqGhTlUNV9r60M2Z5spyWfXW' # This is the fake app's readonly master key
# }

# data = json.loads(requests.get(url, headers=headers,verify=False).content.decode('utf-8')) # Here you have the data that you need
# dfcat = pd.DataFrame(data['results'])
# dfcat.drop(columns = ['objectId','createdAt','updatedAt'], inplace = True)
# for col in ['Make','Model','Category']:
#     dfcat[col] = dfcat[col].apply(lambda x:x.upper())

# dfcat['Category'] = dfcat['Category'].apply(lambda x:'SEDAN' if 'SEDAN' in x else x)
# dfcat['Category'] = dfcat['Category'].apply(lambda x:'COUPE' if 'COUPE' in x else x)
# dfcat['Category'] = dfcat['Category'].apply(lambda x:'HATCHBACK' if 'HATCHBACK' in x else x)
# dfcat['Category'] = dfcat['Category'].apply(lambda x:'WAGON' if 'WAGON' in x else x)
# dfcat['Model'] = dfcat['Model'].apply(lambda x:x.replace('-',''))
# df_category = dfcat[['Year','Make','Model','Category']]
# df_category.drop_duplicates(inplace = True)
# df_category.to_csv("car_category.csv",index = False)

df_category = pd.read_csv('car_category.csv', index_col = False)
df_category.head()

,Year,Make,Model,Category
0,2020,AUDI,Q3,SUV
1,2020,CHEVROLET,MALIBU,SEDAN
2,2020,CADILLAC,ESCALADE ESV,SUV
3,2020,CHEVROLET,CORVETTE,COUPE
4,2020,ACURA,RLX,SEDAN


### Data Set 3 :  Car Reliability Index

<span style="color:red"> #### Dataframe = df_reliability </span>

Read:

    Reliability rankings from consumer review reports in https://www.usatoday.com/story/money/cars/2019/11/14/consumer-reports-auto-reliability-study-2020-vehicles/2578463001/


In [4]:
# Get reliability rankings data

df_reliability = pd.read_csv('car_reliability_rankings.csv')
df_reliability.head()

,ReliabilityRank,Make,Score
0,1,GENESIS,89
1,2,LEXUS,100
2,3,BUICK,103
3,4,PORSCHE,104
4,5,TOYOTA,113


### Data Set 4 :  Cost of Living and Transportation Index by State in the US

<span style="color:red"> ####Dataframe = df_cost </span>

Read:

    Cost Of Living Index(CLI) and Local Purchasing Power Index (LPPI) are socio-economic indicators of a region 
    Data from https://meric.mo.gov/data/cost-living-data-series


In [5]:
# Get economic data

# url = 'https://meric.mo.gov/data/cost-living-data-series'
# response = requests.get(url,verify = False).content.decode('utf-8')
# df_cost = pd.read_html(response)[0] 
# df_cost['State'] = df_cost['State'].str.upper()
# df_cost.drop(df_cost.index[-1],axis = 0)
# states = pd.read_csv("US_States.csv")
# states['StateName'] = states['StateName'].str.upper()
# df_cost = pd.merge(df_cost,states,how = 'inner',left_on='State', right_on='StateName')
# df_cost = df_cost[['StateCode','Rank','Index','Transportation']]
# df_cost.columns = ['State',  'CostOfLivingRank','CostOfLivingIndex','TransportationIndex']
# df_cost.to_csv('statewise_economic_indicators.csv',index = False)

df_cost = pd.read_csv('statewise_economic_indicators.csv',index_col = False)
df_cost.head()

,State,CostOfLivingRank,CostOfLivingIndex,TransportationIndex
0,MS,1,84.70,89.50
1,KS,2,86.50,94.70
2,OK,3,86.70,93.00
3,NM,4,87.20,91.70
4,AR,5,88.20,89.80


### Data Set 5 : % of market sales in 2019/2020 by make

<span style="color:red"> ####Dataframe = df_sales </span>

    Obtained from 
        https://www.goodcarbadcar.net/2020-us-vehicle-sales-figures-by-model 
        https://www.goodcarbadcar.net/2019-us-vehicle-sales-figures-by-model


In [6]:
# Get Car Sales data

# url = 'https://www.goodcarbadcar.net/2020-us-vehicle-sales-figures-by-model/'
# response = requests.get(url,verify = False).content
# dfs = pd.read_html(response) 
# df2020 = dfs[0]
# df2020.dropna(inplace = True)
# df2020 = df2020.apply(pd.to_numeric, errors='ignore') 
# df2020['TotalSales'] = df2020.iloc[:,:7].sum(axis=1)

# url = 'https://www.goodcarbadcar.net/2019-us-vehicle-sales-figures-by-model/'
# response = requests.get(url,verify = False).content
# dfs = pd.read_html(response) 
# df2019 = dfs[1]
# df2019.dropna(inplace = True)
# df2019 = df2019.apply(pd.to_numeric, errors='ignore') 
# df2019['TotalSales'] = df2019.mean(axis=1)

# df_sales = pd.concat([df2020[['Model','TotalSales']], df2019[['Model','TotalSales']]], axis=0)

# df_sales ['Model'] = df_sales ['Model'].str.upper()
# df_sales ['Model'] = df_sales ['Model'].apply(lambda x:x.replace('ALFA ROMEO','ALFAROMEO').replace('LAND ROVER','LANDROVER').replace('ASTON MARTIN','ASTONMARTIN'))
# df_sales ['Make'] = df_sales ['Model'].apply(lambda x:x.split()[0])
# df_sales ['Make'] = df_sales ['Make'].apply(lambda x:x.replace('ALFAROMEO','ALFA ROMEO').replace('LANDROVER','LAND ROVER').replace('ASTONMARTIN','ASTON MARTIN'))
# df_sales = df_sales.groupby('Make').sum().reset_index()


# df_sales['PercentSales'] = (df_sales['TotalSales'] / df_sales['TotalSales'].sum()) * 100
# df_sales.to_csv('car_sales.csv',index = False)

df_sales = pd.read_csv('car_sales.csv',index_col = False)
df_sales.head()

,Make,TotalSales,PercentSales
0,ACURA,69104.42,0.88
1,ALFA ROMEO,8961.50,0.11
2,AUDI,94885.92,1.20
3,BMW,140176.75,1.78
4,BUICK,86634.83,1.10


### Data Set 6 : Used car - days to turn (2016 data)


<span style="color:red"> ####Dataframe = df_turn </span>

Read: 
        
    Days to Turn by Make is the average number of days vehicles were in dealer inventory before being sold during the months indicated.
    
     Obtained from : https://www.edmunds.com/industry-center/data/days-to-turn-by-make.html

In [7]:
df_turn = pd.read_csv("used_car_time_to_turn.csv")
df_turn.head()
df_turn['AvgDaysToTurn'] = df_turn.mean(axis=1)
df_turn['Make'] = df_turn['Make'].str.upper()
df_turn = df_turn[['Make','AvgDaysToTurn']]
df_turn.head()

,Make,AvgDaysToTurn
0,ACURA,65.54
1,AUDI,55.38
2,BMW,74.00
3,BUICK,82.92
4,CADILLAC,80.23


<div class="alert alert-block alert-success">
    
# Data accumulation into single dataframe

    df_cardata =  Main Car data 
    df_category = Category by make and model
    df_reliability = Brand reliability
    df_cost = Cost of living of an area  
    df_sales = Percent Sales by Make
    df_turn = Days to turn by Make
    
    
The objective is to append potentially important fields (for EDA) from the data sets to the main car data set. 
</div>

### <span style="color:Blue"> A : Car Data with Category Data </span>

#### CHALLENGE :  Linking datasets with Make and Model

While "Make" is a standard value for all the above data sets, "Model" values are non standardized and is difficult to be considered as a key for join.

For example:  The model values in df_category seem to be appended with a Trim and hence cannot be joined directly with df_cardata 


In [8]:
d1 = df_cardata[df_cardata['model'].apply(lambda x: len(x.split()) > 1)][['make','model']]
d1[d1['make'] == 'CHEVROLET'].drop_duplicates().head(10)

,make,model
17,CHEVROLET,EXPRESS PASSENGER
95,CHEVROLET,SILVERADO 1500
610,CHEVROLET,MONTE CARLO
640,CHEVROLET,EXPRESS CARGO VAN
1252,CHEVROLET,SILVERADO 1500 LD
1944,CHEVROLET,SILVERADO 2500HD
3028,CHEVROLET,MALIBU MAXX
3509,CHEVROLET,EXPRESS COMMERCIAL CUTAWAY
4762,CHEVROLET,SILVERADO 3500HD
6010,CHEVROLET,SILVERADO 3500HD CHASSIS CAB


In [9]:
d2 = df_category[df_category['Model'].apply(lambda x: len(x.split()) > 1)][['Make','Model']]
d2[d2['Make'] == 'CHEVROLET'].drop_duplicates().head(10)

# There will be no exact match for "SILVERADO 1500" or "EXPRESS PASSENGER" in the df_cardata. 

,Make,Model
5,CHEVROLET,SILVERADO 2500 HD CREW CAB
8,CHEVROLET,COLORADO CREW CAB
11,CHEVROLET,SILVERADO 3500 HD CREW CAB
55,CHEVROLET,SILVERADO 1500 CREW CAB
71,CHEVROLET,BOLT EV
299,CHEVROLET,EXPRESS 2500 CARGO
300,CHEVROLET,COLORADO EXTENDED CAB
301,CHEVROLET,EXPRESS 2500 PASSENGER
302,CHEVROLET,EXPRESS 3500 PASSENGER
306,CHEVROLET,SILVERADO 2500 HD DOUBLE CAB


#### SOLUTION :  

#### A partial word match process would be used to join the data between df_cardata and df_category


In [10]:
def getcat(make,model):
    # get all cars with the input make
    try:
        # get all models listed under the make to reduce search candidates
        df_makemodels = df_category[df_category['Make'] == make]
        # find out the number of words matching between the input model and the existing model values
        set_allmodelvalues = [set(m.split()) for m in df_makemodels.Model.values]
        set_inputmodel = set(model.split())
        wordcomp = np.asarray([len(set_inputmodel.intersection(m)) for m in set_allmodelvalues])
        # find the position of the list for which the model value is maximum overlapping with input model value
        maxpos = np.argmax(wordcomp) if wordcomp.max() >= 1 else -1
        # return the category in that position
        return df_makemodels.iloc[maxpos]['Category'] if maxpos >=0 else None
    except:
        return None

In [11]:
# Test the function with a smaple input. Which category is closet match to 'SILVERADO 1500' for 'CHEVROLET'
getcat('CHEVROLET','SILVERADO 1500')

'PICKUP'

In [12]:
# Another test
getcat('CHEVROLET','EXPRESS PASSENGER')

'VAN/MINIVAN'

In [13]:
# Apply this function to the main data frame
df_cardata['cartype'] = df_cardata.apply(lambda x: getcat(x['make'], x['model']), axis=1)

In [14]:
df_cardata.head()

,vin,year,make,model,trim,pricecategory,price,mileage,city,state,colorexterior,colorinterior,accidenthist,owner,usage,discount,cartype
0,WBY1Z2C51FV286674,2015,BMW,I3,60 AH,FAIR PRICE,15991,21493,BELLEVUE,WA,SILVER,GRAY,1,2,PERSONAL,N,HATCHBACK
1,2GNAXHEV4J6220616,2018,CHEVROLET,EQUINOX,LS WITH 1LS FWD,FAIR PRICE,14899,37071,NORCO,CA,BLACK,GRAY,0,1,PERSONAL,Y,SUV
2,4S3GTAD6XK3741106,2019,SUBARU,IMPREZA,2.0I PREMIUM 5-DOOR CVT,FAIR PRICE,19220,15914,STAFFORD,TX,RED,BEIGE,0,1,PERSONAL,N,SEDAN
3,2C4RDGCG8KR551301,2019,DODGE,GRAND CARAVAN,SXT,EXCELLENT PRICE,12993,42070,OCALA,FL,GRAY,BLACK,0,1,PERSONAL,Y,VAN/MINIVAN
4,5YFEPRAEXLP047434,2020,TOYOTA,COROLLA,LE CVT,EXCELLENT PRICE,13800,18725,BOERNE,TX,WHITE,BLACK,0,1,PERSONAL,N,HATCHBACK


In [15]:
# check if there is any missing data in the new appended field "cartype"
len(df_cardata[df_cardata['cartype'].isnull()])

73

In [16]:
df_cardata[df_cardata['cartype'].isnull()].head()

,vin,year,make,model,trim,pricecategory,price,mileage,city,state,colorexterior,colorinterior,accidenthist,owner,usage,discount,cartype
838,2G4WS52JX31152721,2003,BUICK,CENTURY,CUSTOM,EXCELLENT PRICE,2000,149861,LOVES PARK,IL,WHITE,GRAY,0,3,PERSONAL,Y,None
855,1MEFM66L7XK620196,1999,MERCURY,MYSTIQUE,4DR SEDAN LS,NAN,5000,37641,DURHAM,NC,BLUE,BEIGE,1,1,PERSONAL,N,None
1046,1FDFE4FS1CDA17507,2012,FORD,ECONOLINE COMMERCIAL CUTAWAY,"E-450 158"" DRW",NAN,19988,3540,GAINESVILLE,GA,WHITE,GRAY,1,3,PERSONAL,N,None
1195,1G4HP54K91U205829,2001,BUICK,LESABRE,CUSTOM,NAN,2788,187143,VANCOUVER,OR,GRAY,GRAY,1,7,PERSONAL,N,None
1387,1G4HR52K4WH419155,1998,BUICK,LESABRE,LIMITED,EXCELLENT PRICE,2800,97787,INDIANAPOLIS,IN,WHITE,BEIGE,0,3,PERSONAL,Y,None


#### 73 car data records could not be associated to a category value - the cars seem to be old and no category information is found.
#### but the percentage of such records is low, so this step can be marked as complete.

### <span style="color:Blue"> B : Car Data with Brand Reliability Data </span>

In [17]:
# Add Brand reliability
df_cardata = df_cardata.merge(df_reliability,how = 'left',left_on='make', right_on='Make')
df_cardata.drop(columns = ['Make','Score'],inplace = True)
df_cardata.head()

,vin,year,make,model,trim,pricecategory,price,mileage,city,state,colorexterior,colorinterior,accidenthist,owner,usage,discount,cartype,ReliabilityRank
0,WBY1Z2C51FV286674,2015,BMW,I3,60 AH,FAIR PRICE,15991,21493,BELLEVUE,WA,SILVER,GRAY,1,2,PERSONAL,N,HATCHBACK,8.00
1,2GNAXHEV4J6220616,2018,CHEVROLET,EQUINOX,LS WITH 1LS FWD,FAIR PRICE,14899,37071,NORCO,CA,BLACK,GRAY,0,1,PERSONAL,Y,SUV,9.00
2,4S3GTAD6XK3741106,2019,SUBARU,IMPREZA,2.0I PREMIUM 5-DOOR CVT,FAIR PRICE,19220,15914,STAFFORD,TX,RED,BEIGE,0,1,PERSONAL,N,SEDAN,23.00
3,2C4RDGCG8KR551301,2019,DODGE,GRAND CARAVAN,SXT,EXCELLENT PRICE,12993,42070,OCALA,FL,GRAY,BLACK,0,1,PERSONAL,Y,VAN/MINIVAN,25.00
4,5YFEPRAEXLP047434,2020,TOYOTA,COROLLA,LE CVT,EXCELLENT PRICE,13800,18725,BOERNE,TX,WHITE,BLACK,0,1,PERSONAL,N,HATCHBACK,5.00


In [18]:
# check if there is any missing data in the new appended field "ReliabilityRank"
len(df_cardata[df_cardata['ReliabilityRank'].isnull()])

79

In [19]:
df_cardata[df_cardata['ReliabilityRank'].isnull()]['make'].unique()

array(['HUMMER', 'SCION', 'ISUZU', 'MERCURY', 'PONTIAC', 'SATURN',
       'MASERATI', 'SMART', 'MAYBACH', 'BENTLEY', 'ASTON MARTIN', 'SAAB',
       'SUZUKI', 'TESLA'], dtype=object)

#### 362 records of 17 brands didn't have a reliability rank, these are either non-US makers or pickup/luxury brands which were beyond the rankings

### <span style="color:Blue"> C : Car Data with Socio Economic indicators for the area (state) where car is listed </span>

In [20]:
df_cardata = df_cardata.merge(df_cost,how = 'left',left_on=['state'], right_on=['State'])
df_cardata.drop(columns = ['State'],inplace = True)
df_cardata.head()

,vin,year,make,model,trim,pricecategory,price,mileage,city,state,colorexterior,colorinterior,accidenthist,owner,usage,discount,cartype,ReliabilityRank,CostOfLivingRank,CostOfLivingIndex,TransportationIndex
0,WBY1Z2C51FV286674,2015,BMW,I3,60 AH,FAIR PRICE,15991,21493,BELLEVUE,WA,SILVER,GRAY,1,2,PERSONAL,N,HATCHBACK,8.00,39,112.20,118.60
1,2GNAXHEV4J6220616,2018,CHEVROLET,EQUINOX,LS WITH 1LS FWD,FAIR PRICE,14899,37071,NORCO,CA,BLACK,GRAY,0,1,PERSONAL,Y,SUV,9.00,49,142.70,133.00
2,4S3GTAD6XK3741106,2019,SUBARU,IMPREZA,2.0I PREMIUM 5-DOOR CVT,FAIR PRICE,19220,15914,STAFFORD,TX,RED,BEIGE,0,1,PERSONAL,N,SEDAN,23.00,14,91.70,91.10
3,2C4RDGCG8KR551301,2019,DODGE,GRAND CARAVAN,SXT,EXCELLENT PRICE,12993,42070,OCALA,FL,GRAY,BLACK,0,1,PERSONAL,Y,VAN/MINIVAN,25.00,27,99.20,98.70
4,5YFEPRAEXLP047434,2020,TOYOTA,COROLLA,LE CVT,EXCELLENT PRICE,13800,18725,BOERNE,TX,WHITE,BLACK,0,1,PERSONAL,N,HATCHBACK,5.00,14,91.70,91.10


In [21]:
# check if there is any missing data in the new appended field "CostOfLivingRank"
len(df_cardata[df_cardata['CostOfLivingRank'].isnull()])

0


### <span style="color:Blue"> D : Car Data with Percent Sales by Make in the US </span>

In [22]:
df_cardata = df_cardata.merge(df_sales,how = 'left',left_on=['make'], right_on=['Make'])
df_cardata.drop(columns = ['Make','TotalSales'],inplace = True)
df_cardata.rename(columns = {'PercentSales':'BrandPercentSales'},inplace = True)
df_cardata.head()

,vin,year,make,model,trim,pricecategory,price,mileage,city,state,colorexterior,colorinterior,accidenthist,owner,usage,discount,cartype,ReliabilityRank,CostOfLivingRank,CostOfLivingIndex,TransportationIndex,BrandPercentSales
0,WBY1Z2C51FV286674,2015,BMW,I3,60 AH,FAIR PRICE,15991,21493,BELLEVUE,WA,SILVER,GRAY,1,2,PERSONAL,N,HATCHBACK,8.00,39,112.20,118.60,1.78
1,2GNAXHEV4J6220616,2018,CHEVROLET,EQUINOX,LS WITH 1LS FWD,FAIR PRICE,14899,37071,NORCO,CA,BLACK,GRAY,0,1,PERSONAL,Y,SUV,9.00,49,142.70,133.00,11.69
2,4S3GTAD6XK3741106,2019,SUBARU,IMPREZA,2.0I PREMIUM 5-DOOR CVT,FAIR PRICE,19220,15914,STAFFORD,TX,RED,BEIGE,0,1,PERSONAL,N,SEDAN,23.00,14,91.70,91.10,4.13
3,2C4RDGCG8KR551301,2019,DODGE,GRAND CARAVAN,SXT,EXCELLENT PRICE,12993,42070,OCALA,FL,GRAY,BLACK,0,1,PERSONAL,Y,VAN/MINIVAN,25.00,27,99.20,98.70,2.13
4,5YFEPRAEXLP047434,2020,TOYOTA,COROLLA,LE CVT,EXCELLENT PRICE,13800,18725,BOERNE,TX,WHITE,BLACK,0,1,PERSONAL,N,HATCHBACK,5.00,14,91.70,91.10,12.19


In [23]:
# check if there is any missing data in the new appended field "BrandPercentSales"
len(df_cardata[df_cardata['CostOfLivingRank'].isnull()])

0

In [24]:
dfc = df_cardata.copy()


### <span style="color:Blue"> E : Car Data with typical "Days to Turn" value for used cars (2016 data) </span>

In [25]:
df_cardata = df_cardata.merge(df_turn,how = 'left',left_on=['make'], right_on=['Make'])
df_cardata.drop(columns = ['Make'],inplace = True)
df_cardata.rename(columns = {'AvgDaysToTurn':'BrandAvgDaysToTurn'},inplace = True)
df_cardata.head()

,vin,year,make,model,trim,pricecategory,price,mileage,city,state,colorexterior,colorinterior,accidenthist,owner,usage,discount,cartype,ReliabilityRank,CostOfLivingRank,CostOfLivingIndex,TransportationIndex,BrandPercentSales,BrandAvgDaysToTurn
0,WBY1Z2C51FV286674,2015,BMW,I3,60 AH,FAIR PRICE,15991,21493,BELLEVUE,WA,SILVER,GRAY,1,2,PERSONAL,N,HATCHBACK,8.00,39,112.20,118.60,1.78,74.00
1,2GNAXHEV4J6220616,2018,CHEVROLET,EQUINOX,LS WITH 1LS FWD,FAIR PRICE,14899,37071,NORCO,CA,BLACK,GRAY,0,1,PERSONAL,Y,SUV,9.00,49,142.70,133.00,11.69,78.23
2,4S3GTAD6XK3741106,2019,SUBARU,IMPREZA,2.0I PREMIUM 5-DOOR CVT,FAIR PRICE,19220,15914,STAFFORD,TX,RED,BEIGE,0,1,PERSONAL,N,SEDAN,23.00,14,91.70,91.10,4.13,23.23
3,2C4RDGCG8KR551301,2019,DODGE,GRAND CARAVAN,SXT,EXCELLENT PRICE,12993,42070,OCALA,FL,GRAY,BLACK,0,1,PERSONAL,Y,VAN/MINIVAN,25.00,27,99.20,98.70,2.13,98.54
4,5YFEPRAEXLP047434,2020,TOYOTA,COROLLA,LE CVT,EXCELLENT PRICE,13800,18725,BOERNE,TX,WHITE,BLACK,0,1,PERSONAL,N,HATCHBACK,5.00,14,91.70,91.10,12.19,43.23


<div class="alert alert-block alert-success">
    
# COMPLETED Data Set Creation
    
</div>

In [26]:
df_cardata

,vin,year,make,model,trim,pricecategory,price,mileage,city,state,colorexterior,colorinterior,accidenthist,owner,usage,discount,cartype,ReliabilityRank,CostOfLivingRank,CostOfLivingIndex,TransportationIndex,BrandPercentSales,BrandAvgDaysToTurn
0,WBY1Z2C51FV286674,2015,BMW,I3,60 AH,FAIR PRICE,15991,21493,BELLEVUE,WA,SILVER,GRAY,1,2,PERSONAL,N,HATCHBACK,8.00,39,112.20,118.60,1.78,74.00
1,2GNAXHEV4J6220616,2018,CHEVROLET,EQUINOX,LS WITH 1LS FWD,FAIR PRICE,14899,37071,NORCO,CA,BLACK,GRAY,0,1,PERSONAL,Y,SUV,9.00,49,142.70,133.00,11.69,78.23
2,4S3GTAD6XK3741106,2019,SUBARU,IMPREZA,2.0I PREMIUM 5-DOOR CVT,FAIR PRICE,19220,15914,STAFFORD,TX,RED,BEIGE,0,1,PERSONAL,N,SEDAN,23.00,14,91.70,91.10,4.13,23.23
3,2C4RDGCG8KR551301,2019,DODGE,GRAND CARAVAN,SXT,EXCELLENT PRICE,12993,42070,OCALA,FL,GRAY,BLACK,0,1,PERSONAL,Y,VAN/MINIVAN,25.00,27,99.20,98.70,2.13,98.54
4,5YFEPRAEXLP047434,2020,TOYOTA,COROLLA,LE CVT,EXCELLENT PRICE,13800,18725,BOERNE,TX,WHITE,BLACK,0,1,PERSONAL,N,HATCHBACK,5.00,14,91.70,91.10,12.19,43.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,3N1AB7AP3KY447078,2019,NISSAN,SENTRA,S CVT,EXCELLENT PRICE,13995,2816,DALLAS,TX,BLACK,BLACK,0,1,PERSONAL,N,SEDAN,16.00,14,91.70,91.10,6.29,76.23
9992,2T3RFREV4JW789216,2018,TOYOTA,RAV4,XLE AWD,EXCELLENT PRICE,18495,43908,MONROE,NC,SILVER,UNKNOWN,0,1,PERSONAL,Y,SUV,5.00,22,95.50,93.60,12.19,43.23
9993,1N6AA0EJ9FN509080,2015,NISSAN,TITAN,SV CREW CAB 4WD,EXCELLENT PRICE,13000,139839,ARLINGTON,TX,SILVER,BLACK,0,1,FLEET,N,PICKUP,16.00,14,91.70,91.10,6.29,76.23
9994,1FTEW1EP5FKE37943,2015,FORD,F150,XL SUPERCREW 5.5' BOX 4WD,EXCELLENT PRICE,19900,93501,COMANCHE,TX,WHITE,GRAY,0,1,PERSONAL,N,PICKUP,10.00,14,91.70,91.10,13.83,71.62


In [27]:
len(df_cardata)

9996

### <span style="color:Blue"> Bonus data addition - Used Car Ratings </span>

Data Source : https://cars.usnews.com/


In [28]:
df_ratings = pd.read_csv('car_ratings.csv', index_col = False)
df_ratings[['Make','Model']] = df_ratings['MakeModel'].str.split(" ",expand=True,n=1)
df_ratings.drop_duplicates(subset = ['Make','Model'], inplace = True)
df_ratings['AvgMPG'] = (df_ratings['MpgCity']  + df_ratings['MpgHwy']) / 2
df_ratings = df_ratings[['Make','Model','CarClass','ReviewScore','AvgMPG']]
df_ratings.head()

,Make,Model,CarClass,ReviewScore,AvgMPG
0,HONDA,FIT,AFFORDABLE SMALL CARS,8.70,36.50
1,VOLKSWAGEN,GTI,AFFORDABLE SMALL CARS,8.60,28.00
2,CHEVROLET,CRUZE,AFFORDABLE SMALL CARS,8.50,33.00
3,KIA,SOUL,AFFORDABLE SMALL CARS,8.50,28.50
4,CHEVROLET,SONIC,AFFORDABLE SMALL CARS,8.40,30.00


In [29]:
df_ratings[df_ratings.duplicated(subset = ['Make','Model'])]

,Make,Model,CarClass,ReviewScore,AvgMPG


In [30]:
df_cardata = pd.merge(df_cardata, df_ratings, how = 'left',  left_on = ['make','model'], right_on = ['Make','Model'])
df_cardata.drop(columns = ['Make','Model'],inplace = True)
df_cardata

,vin,year,make,model,trim,pricecategory,price,mileage,city,state,colorexterior,colorinterior,accidenthist,owner,usage,discount,cartype,ReliabilityRank,CostOfLivingRank,CostOfLivingIndex,TransportationIndex,BrandPercentSales,BrandAvgDaysToTurn,CarClass,ReviewScore,AvgMPG
0,WBY1Z2C51FV286674,2015,BMW,I3,60 AH,FAIR PRICE,15991,21493,BELLEVUE,WA,SILVER,GRAY,1,2,PERSONAL,N,HATCHBACK,8.00,39,112.20,118.60,1.78,74.00,LUXURY HYBRID CARS,7.50,113.00
1,2GNAXHEV4J6220616,2018,CHEVROLET,EQUINOX,LS WITH 1LS FWD,FAIR PRICE,14899,37071,NORCO,CA,BLACK,GRAY,0,1,PERSONAL,Y,SUV,9.00,49,142.70,133.00,11.69,78.23,AFFORDABLE COMPACT SUVS,8.40,29.00
2,4S3GTAD6XK3741106,2019,SUBARU,IMPREZA,2.0I PREMIUM 5-DOOR CVT,FAIR PRICE,19220,15914,STAFFORD,TX,RED,BEIGE,0,1,PERSONAL,N,SEDAN,23.00,14,91.70,91.10,4.13,23.23,AFFORDABLE SMALL CARS,7.70,33.00
3,2C4RDGCG8KR551301,2019,DODGE,GRAND CARAVAN,SXT,EXCELLENT PRICE,12993,42070,OCALA,FL,GRAY,BLACK,0,1,PERSONAL,Y,VAN/MINIVAN,25.00,27,99.20,98.70,2.13,98.54,MINIVANS,6.80,21.00
4,5YFEPRAEXLP047434,2020,TOYOTA,COROLLA,LE CVT,EXCELLENT PRICE,13800,18725,BOERNE,TX,WHITE,BLACK,0,1,PERSONAL,N,HATCHBACK,5.00,14,91.70,91.10,12.19,43.23,HATCHBACKS,7.90,32.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,3N1AB7AP3KY447078,2019,NISSAN,SENTRA,S CVT,EXCELLENT PRICE,13995,2816,DALLAS,TX,BLACK,BLACK,0,1,PERSONAL,N,SEDAN,16.00,14,91.70,91.10,6.29,76.23,AFFORDABLE SMALL CARS,7.20,33.00
9992,2T3RFREV4JW789216,2018,TOYOTA,RAV4,XLE AWD,EXCELLENT PRICE,18495,43908,MONROE,NC,SILVER,UNKNOWN,0,1,PERSONAL,Y,SUV,5.00,22,95.50,93.60,12.19,43.23,AFFORDABLE COMPACT SUVS,7.90,30.50
9993,1N6AA0EJ9FN509080,2015,NISSAN,TITAN,SV CREW CAB 4WD,EXCELLENT PRICE,13000,139839,ARLINGTON,TX,SILVER,BLACK,0,1,FLEET,N,PICKUP,16.00,14,91.70,91.10,6.29,76.23,FULL SIZE TRUCKS,7.70,18.00
9994,1FTEW1EP5FKE37943,2015,FORD,F150,XL SUPERCREW 5.5' BOX 4WD,EXCELLENT PRICE,19900,93501,COMANCHE,TX,WHITE,GRAY,0,1,PERSONAL,N,PICKUP,10.00,14,91.70,91.10,13.83,71.62,NaN,nan,nan


In [31]:
df_cardata.to_csv('cardata_final.csv',index = False)